<a href="https://colab.research.google.com/github/YoungHyunKoo/GEE_remote_sensing/blob/main/2025_extra_Weddell_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Sea ice project for the Weddell Sea (for Kuba)**
# **Find and download image collections**

### OBJECTIVES
1. Explore how an image collection is constructed.
2. Use spatial and temporal filter functions to ge the image collection of interest.
3. Visualize image collections.
4. Download images as GeoTIFF files in local PC.

Credited by Younghyun Koo (kooala317@gmail.com); Last update on Feb 7, 2025




### **Import necessary librarires**

First, let's import and initialize `ee` library.


In [1]:
# Import ee library
import ee

# Authenticate
ee.Authenticate()

# Initialize with your own project - Please specify your own project name.
ee.Initialize(project = "utsa-spring2024")

In [126]:
%pip install geedim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 49.7 MB/s eta 0:00:00


In [127]:
import geemap
import glob, os
import geedim

### **Import Sentinel-2 image collection**

We will import [Sentinel-2 MSI: MultiSpectral Instrument, Level-2A](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED) image collection.

In [3]:
# Import image collection - Landsat 8 surface reflectance
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

### **Filter by dates**

There is a function named `filterDate` to filter an image collection by a date range. You need to provide start and end day for this function. You can find the details of `filterDate` function here: [filterDate](https://developers.google.com/earth-engine/apidocs/ee-imagecollection-filterdate)

In [ ]:
# ImageCollection.filterDate(start, end)
collction = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filterDate('2020-05-01', '2020-06-01')

**NOTE:** Start date is INCLUSIVE but end date is EXCLUSIVE.

### **Filter by location**

In addition to the temporal filtering, you can use the `filterBounds` function to narrow down to your region of interest (ROI). You will filter out the image collection based on the GEE `geometry`. Before running the `filterBounds` function, you need to define the `geometry` as a ROI.

In [59]:
# Region of interest as a point (longitude, latitude)
west = -60
east = -15
north = -65
south = -78
roi = ee.Geometry.BBox(west, south, east, north)

roi = ee.Geometry.Polygon([[-58.419317087047574,-65.04394184910552],
 [-59.561895212047574,-74.6655831083171],
 [-44.29057486986493,-77.60132178367095],
  [-28.997606119864933,-75.95503083864251],
   [-25.745652994864933,-73.82802897726148],
    [-17.220262369864937,-70.79070022226655],
     [-58.419317087047574,-65.04394184910552]])

In [60]:
Map = geemap.Map()
Map.addLayer(roi, {}, "Line example")
Map.centerObject(roi)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

### **Filter by metadata**

As you can see in the image properties above, this image is from May 10, 2020. (See "system:time_end") However, the cloud cover of this image is about 50 % (i.e., 50 % of the entire image is covered by cloud). This cloud-covered image is not very useful for monitoring earth surfaces. Therefore, we need to select some images with low cloud covers. GEE provides the function named `filterMetadata` to filter out the image collection based on the metadata. To filter out cloud-covered images, we will use the 'CLOUD_COVER' property from the metadata.

In [132]:
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
    .filterDate('2018-10-01', '2019-03-30') \
    .filterBounds(roi) \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20) \
    .sort("CLOUDY_PIXEL_PERCENTAGE")

# Filter by metadata: cloud cover is less than 10 %.
# In this case, we use 'less than' operator to filter out low cloud cover area.
# However, there are some other operators as well: 'greater than', 'equals', etc.
# .sort function is to sort the images in the image collection based on cloud covers.
# That is, the images are sorted from low cloud covers to higher cloud covers


In [133]:
# Number of images in the filtered image collection
print(collection.size().getInfo())

354


In [134]:
# Load a geemap
Map = geemap.Map()

# "First" function gets the first image in the collection
image = collection #.first()

# image visualization factors
vis_param = {'min': 0,
             'max': 10000,
             'bands': ['B4', 'B3', 'B2'],
             'gamma': 1.0}

# Number of cloud-free pixels
not_cloud_col = collection.map(
    lambda img: img.updateMask(img.select('MSK_CLDPRB').lte(10))
)
count = not_cloud_col.count()

# Visualize the number of cloud-free pixels
Map.add_layer(count, {'min': 1, 'max': 5}, 'Count (not cloud observations)')

# Visualize available images
Map.addLayer(image, vis_param, "First image")
Map.centerObject(image, 6)

Map

Map(center=[-73.8247113598815, -49.03610330169089], controls=(WidgetControl(options=['position', 'transparent_…

In [135]:
# Please draw a point at the map below
center = Map.draw_last_feature.getInfo()['geometry']['coordinates']

In [136]:
# Set a ~100 km extent for image downloading
extent = ee.Geometry.BBox(center[0]-2, center[1]-0.5, center[0]+2, center[1]+0.5)

Map = geemap.Map()
Map.addLayer(extent, {}, "Line example")
Map.centerObject(extent)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [129]:
# Google drive directory to store the images (you can skip if you want to download images to your local PC)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [131]:
def add_coverage(img, extent = extent):
    tol = 1000
    overlap = img.geometry().intersection(extent, tol)
    ratio = overlap.area(tol).divide(extent.area(tol))
    return img.set({'coverage_ratio': ratio})

# calculate coverage area of image to roi
def coverage(img, extent = extent):
    tol = 1000
    overlap = img.geometry().intersection(extent, tol)
    ratio = overlap.area(tol).divide(extent.area(tol))
    return ratio.getInfo()

# Calculate coverage area for the given target extent
collection = collection.map(add_coverage).filter(ee.Filter.gt('coverage_ratio', 0.7))

S2_ids = collection.aggregate_array('system:id').getInfo()

for k in S2_ids:

    bands = ["B5", "B4", "B3"]
    img = ee.Image(k)
    # Coordinated reference system (geocoordination)
    crs = img.getInfo()['bands'][0]['crs']

    img = img.select(bands)

    # Where to save the images
    # If you run this code on your local machine, you should change it to your target directory accordingly.
    out_dir = "drive/MyDrive/Weddell"

    r = img.get('coverage_ratio').getInfo()

    name = os.path.basename(k)
    description = f"S2_{name}"
    print(description)

    # Download images to the target directory
    geemap.download_ee_image_tiles(
        img, ee.FeatureCollection(extent), out_dir, prefix=f"{description}_", crs=crs, scale=20
    )


S2_20190130T120239_20190130T120241_T21CWT


S2_20190130T120239_20190130T120241_T21CWT_1.tif: |          | 0.00/216M (raw) [  0.0%] in 00:00 (eta:     ?)

Downloaded 1 tiles in 35.69717192649841 seconds.
S2_20190120T120239_20190120T120241_T21CWT


S2_20190120T120239_20190120T120241_T21CWT_1.tif: |          | 0.00/216M (raw) [  0.0%] in 00:00 (eta:     ?)

Downloaded 1 tiles in 35.300572872161865 seconds.


## **References**
- https://geemap.org/tutorials/
- https://developers.google.com/earth-engine/guides/ic_reducing